<a href="https://colab.research.google.com/github/Jin-ahBak/Pytorch-Study/blob/master/Seq2Seq%2B_Attention_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive

In [27]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import pandas as pd

# 기계 번역 데이터 전처리

* Seq2Seq에는 **인코더의 입력, 디코더의 입력, 
디코더의 출력** 에 해당하는 데이터가 필요
*   목표 데이터에는 시작과 끝을 나타내는 토큰이 포함되어야 함
*   여기서는 '\t'와 '\n'을 각각 시작과 끝을 나타내는 토큰으로 사용


In [29]:
lines = pd.read_csv('/content/drive/MyDrive/fra.txt', names = ['src', 'tar', 'lic'], sep='\t')
del lines['lic']  
len(lines)

190206

In [30]:
lines = lines.loc[:, 'src': 'tar']  

lines = lines[0:60000]  

lines.tar = lines.tar.apply(lambda x:'\t' + x + '\n')    

lines[:10]
lines.sample(10)

,src,tar
18285,I like traveling.,\tJ'aime voyager.\n
33996,He accepted the job.,\tIl a accepté le job.\n
52265,Whose handbag is this?,\tÀ qui est ce sac à main ?\n
48168,I helped fix the leak.,\tJ'ai aidé à réparer la fuite.\n
23843,I will do my best.,\tJe vais faire de mon mieux.\n
25769,Tom fed the horse.,\tTom donna à manger au cheval.\n
4093,What a drag!,\tQuel boulet !\n
51744,We were all disgusted.,\tNous étions toutes dégoûtées.\n
20099,They were killed.,\tIls ont été assassinés.\n
2038,I'm a twin.,\tJ'ai un jumeau.\n




*   해당 예제에서는 글자 단위로 예측, 따라서 글자 집합을 구축해주어야 함
*   구축한 다음, 정렬해 인덱스를 부여해 글자에 해당하는 사전을 만듬
*   사전은 글자를 모델에 투입하도록 변환하거나 예측시 반환되는 인덱스들을 글자로 변환할 때 사용



In [31]:
##### 글자 단위로 예측하기에 글자집합 구축 

src_vocab = set()  
for line in lines.src: # 1줄씩 읽음 
    for char in line:  # 1개의 글자씩 읽음 
        src_vocab.add(char)  
        

tar_vocab = set()  
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)  
        

In [32]:
###### 정렬해 인덱스를 부여함 
src_vocab = sorted(list(src_vocab))  
tar_vocab = sorted(list(tar_vocab))  

src_vocab_size = len(src_vocab) + 1 
tar_vocab_size = len(tar_vocab) + 1   


#### 딕셔너리 형태로 만듦.
src_to_idx = dict([(word, i+1) for i, word in enumerate(src_vocab)])  
print(src_to_idx)  

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}


In [33]:
tar_to_idx = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

print(tar_to_idx)  

{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'Z': 52, 'a': 53, 'b': 54, 'c': 55, 'd': 56, 'e': 57, 'f': 58, 'g': 59, 'h': 60, 'i': 61, 'j': 62, 'k': 63, 'l': 64, 'm': 65, 'n': 66, 'o': 67, 'p': 68, 'q': 69, 'r': 70, 's': 71, 't': 72, 'u': 73, 'v': 74, 'w': 75, 'x': 76, 'y': 77, 'z': 78, '\xa0': 79, '«': 80, '»': 81, 'À': 82, 'Ç': 83, 'É': 84, 'Ê': 85, 'Ô': 86, 'à': 87, 'â': 88, 'ç': 89, 'è': 90, 'é': 91, 'ê': 92, 'ë': 93, 'î': 94, 'ï': 95, 'ô': 96, 'ù': 97, 'û': 98, 'œ': 99, '\u2009': 100, '\u200b': 101, '‘': 102, '’': 103, '\u202f': 104}


*   인코더에 입력될 입력 데이터를 구성
*   문장의 글자 하나씩을 사전을 이용해 인덱스로 변환해 리스트에 넣음



In [34]:
### 인코더 입력값  

encoder_input = []  
for line in lines.src: # 입력데이터에서 1줄씩 문장을 읽음 
    encoder_input.append([src_to_idx[w] for w in line]) # 각 줄에서 1개씩 문자 읽고 글자에 해당되는 정수로 변환 
    
print(encoder_input[:5])
    

[[30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10], [31, 58, 10]]




*   디코더에 입력될 입력 데이터를 구성
*   인코더 입력 데이터 처리와 동일하나, 목표 데이터에 해당하는 사전을 사용해주어야 함



In [35]:
### 디코더 입력값 

decoder_input = []  
for line in lines.tar:
    decoder_input.append([tar_to_idx[w] for w in line])
    
print(decoder_input[:5])

[[1, 48, 53, 3, 4, 2], [1, 39, 53, 70, 55, 60, 57, 14, 2], [1, 28, 67, 73, 59, 57, 3, 4, 2], [1, 45, 53, 64, 73, 72, 3, 4, 2], [1, 45, 53, 64, 73, 72, 14, 2]]



*   디코더의 출력과 비교할 목표 데이터를 구성
*   디코더의 입력 데이터를 구성할 때와 동일하나, 시작 토큰을 제외해주어야 함



In [36]:
### 디코더 실제값 = 목표 데이터 = 디코더의 입력데이터와 동일(단, 시작토큰 \t 제외)

decoder_target = []  
for line in lines.tar:
    decoder_target.append([tar_to_idx[w] for w in line if w != '\t'])
    
print(decoder_target[:5])

[[48, 53, 3, 4, 2], [39, 53, 70, 55, 60, 57, 14, 2], [28, 67, 73, 59, 57, 3, 4, 2], [45, 53, 64, 73, 72, 3, 4, 2], [45, 53, 64, 73, 72, 14, 2]]




* 각각의 데이터를 동일한 길이로 맞춰줌
* 길이를 맞춰줄 때는 해당 데이터의 최대 길이로 맞춰줌
* 원 핫 인코딩을 통해 원 핫 벡터로 변환



In [37]:
##### 최대길에 맞춰서 패딩( 데이터를 동일한 길이로 맞춰줌)

from tensorflow.keras.preprocessing.sequence import pad_sequences  

max_src_len = max([len(line) for line in lines.src])  
max_tar_len = max([len(line) for line in lines.tar])  

encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [38]:
## categorical한 one-hot-vector로 변경 

from tensorflow.keras.utils import to_categorical  

encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

# Attention Mechanism




* Attention Mechanism은 디코더가 예측하는 시점마다 인코더의 전체 입력 문장을 다시 한번 참조
* 이때 전체 입력 문장을 단순히 참조하지 않고, **예측할 단어와 연관이 있는 단어를 집중(Attention)**해서 참조

## Attention Mechanism 종류

* attention mechanism에는 스코어 계산 방식의 차이에 따라 다양한 종류가 존재



이 름                 | 스 코 어&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                 
----------------------|-----------------------
dot|$score(s_t, h_i) = s_t^Th_i$
scaled dot|$score(s_t, h_i) = \frac{s_t^Th_i}{\sqrt n}$
general|$score(s_t, h_i) = s_t^TW_ah_i$
concat|$score(s_t, h_i) = W_a^Ttanh(W_b[s_t;h_i])$
location-base|$a_t = softmax(W_as_t)$

  + $s_t$ : querys(t 시점에서의 디코더 셀의 은닉 상태)
  + $h_i$ : keys(모든 시점의 인코더 셀 은닉 상태)
  + $W_a, W_b$ : 학습 가능한 가중치 행렬



## Attention Mechanism 과정

* Attention Mechanism 과정
  1. encoder/ decoder Embedding과 LSTM 레이어 생성
  1. attention score 계산
  2. 소프트맥스 함수를 통한 attention distribution 계산
  3. 각 인코더의 (어텐션 가중치)와 (은닉 상태)를 가중합하여= 내적하여 어텐션 값 계산
  4. (어텐션 값)과 (디코더의 t 시점의 은닉 상태)를 연결
  5. 출력층 연산의 입력이 되는 $\tilde{s}_t$ 계산



### Attention score 계산

### 소프트 맥스 함수를 통한 attention distribution 계산

### 각 인코더의 어텐션 가중치와 은닉 상태를 가중합해 어텐션 값 계산

### 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결

![전체구조](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FIcOuI%2Fbtq9eM78HJe%2Fu6WeljAakJGQCwJvzfIHPK%2Fimg.png)

## Attention Mechanism 모델

### 인코더(Encoder))



*   인코더는 seq2seq에서 작성한 것과 동일



In [39]:
from keras.layers import Input, LSTM  


# encoding에  return_sequences= True추가( 인코더 내부상태를 디코더로 넘겨줌) 
# LSTM의 은닉 상태 크기는 256로 선택
encoder_inputs = Input(shape=(None, src_vocab_size))  
encoder_lstm = LSTM(256, return_state= True)  

# LSTM은 state가 2개: hidden과 cell state 를 디코더로 전달
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)  
encoder_states = [state_h, state_c]

-- hidden state는 계층의 출력 / 다음 타임 step 으로 넘기는 정보  
-- cell state는 기억을 오랫동안 유지할 수 있는 구조/ 새로운 특징을 덧셈으로 받는 구조

![image.png](attachment:image.png)

그렇게 하면 Encoder의 LSTM은 I, LOVE, YOU 3단어를 입력받고 3개의 값을 출력한다.

그리고 Decoder의 LSTM에게 hidden state와, cell state를 넘겨준다.

Encoder에서 넘긴 state들은 encoder에서 압축된 "I love you"의 문맥과 의미 정보를 Decoder에 제공한다.



### 디코더(Decoder) + Attention



*   디코더에서는 seq2seq와는 다르게 **attention layer를 추가함**
*   S_는 은닉 상태와 디코더의 최종 출력을 연결한 결과, 연결할 때 형상을 맞춰주기 위해 축을 추가함
*   attention layer는 디코더의 은닉 상태와 인코더 은닉 상태 전체를 받아 컨텍스트 벡터를 생성함
*   이 때 attention layer는 앞서 설명한 과정 중 1~3번째를 수행, 나머지는 사용자가 연결해주어야 함
*   마지막으로 생성한 컨텍스트 벡터와 디코더의 은닉 상태 전체를 이어 softmax layer에 투입, 인덱스를 예측함



![image.png](attachment:image.png)

In [40]:
import tensorflow as tf  
# Decoder 내부또한 Embedding 레이어와 LSTM레이어로 이루어짐.

decoder_inputs = Input(shape=(None, tar_vocab_size))  
decoder_lstm = LSTM(256, return_sequences = True, return_state= True)    
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states) 


# concat해주는 S_ : hidden과 디코더 output 합쳐야함.  
# score 계산하는 부분 히든레이어에 decoder outputs

S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis= 1)  

In [41]:
from keras.layers import Dense
from keras.layers import Attention

# 어텐션 
attention = Attention()

# 어텐션 값 = (어텐션 가중치 * 인코더의 결과값( 은닉상태))  = context_vector  
context_vector = attention([S_, encoder_outputs])  

# context_vector와 디코더의 은닉상태 연결 
concat = tf.concat([decoder_outputs, context_vector], axis= -1)

# softmax layer에 투입하여 인덱스 예측 
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')  
decoder_outputs = decoder_softmax_layer(concat)

### 모델 구성 및 학습



*   구성하는 방법은 seq2seq와 동일함
*   **attention mechanism을 활용해 학습 시간이 절반 가량 준 것을 확인할 수 있음**



In [42]:
from keras.models import Model  

# 모델생성
model= Model([encoder_inputs, decoder_inputs], decoder_outputs)  
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',  metrics=['acc'])   # 옵티마이저와 손실함수 선택 

# 모델검증
model.fit(x=[encoder_input, decoder_input], y= decoder_target,
         batch_size= 128,
          epochs= 25,
          validation_split = 0.2)

Epoch 1/25
375/375 [==============================] - 29s 69ms/step - loss: 0.8959 - acc: 0.7629 - val_loss: 0.8169 - val_acc: 0.7603
Epoch 2/25
375/375 [==============================] - 25s 66ms/step - loss: 0.5518 - acc: 0.8372 - val_loss: 0.6428 - val_acc: 0.8080
Epoch 3/25
375/375 [==============================] - 25s 66ms/step - loss: 0.4595 - acc: 0.8625 - val_loss: 0.5710 - val_acc: 0.8293
Epoch 4/25
375/375 [==============================] - 25s 66ms/step - loss: 0.4051 - acc: 0.8783 - val_loss: 0.5217 - val_acc: 0.8442
Epoch 5/25
375/375 [==============================] - 25s 65ms/step - loss: 0.3690 - acc: 0.8887 - val_loss: 0.4863 - val_acc: 0.8549
Epoch 6/25
375/375 [==============================] - 24s 65ms/step - loss: 0.3460 - acc: 0.8954 - val_loss: 0.4621 - val_acc: 0.8611
Epoch 7/25
375/375 [==============================] - 25s 66ms/step - loss: 0.3245 - acc: 0.9017 - val_loss: 0.4430 - val_acc: 0.8676
Epoch 8/25
375/375 [==============================] - 25s 65ms

### 예측


###  전체적인 번역 동작단계 
1. 번역하고자 하는 입력문장이 encoder에 들어가 hidden state 와 cell state 얻음
2. state와 <SOS> 에 해당하는 '\t'를 decoder로 보냄
3. decoder가 <EOS> 에 해당하는 '\n'이 나올떄까지 다음 문자 예측 행동 반복 

    
*   예측도 seq2seq와 동일하나, 추가된 모델 구조를 반영해주어야 함(attention layer)
*   encoder와 decoder를 분리해주었기 때문에 디코더에서 인코더의 은닉 상태(estate_h)=( encoder states)와 최종 은닉 상태(encoder_outputs)를 따로 입력받아야 함


In [43]:
# 인코더 모델
encoder_model = Model(inputs= encoder_inputs, 
                      outputs = [encoder_outputs, encoder_states])


# 인코더 디코더 input , output 값 변수생성 
decoder_state_input_h = Input(shape=(256))  
decoder_state_input_c = Input(shape=(256)) 
estate_h = Input(shape=(256))  
encoder_outputs = Input(shape=(256))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]  
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)  
decoder_states = [state_h, state_c]  
S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis= 1)  
context_vector = attention([S_, encoder_outputs]) 
decoder_concat = tf.concat([decoder_outputs, context_vector], axis= -1)  
decoder_outputs = decoder_softmax_layer(decoder_concat)  


# 디코더 모델 
decoder_model = Model(inputs=[decoder_inputs, estate_h, encoder_outputs] + decoder_state_inputs,
                     outputs=[decoder_outputs]+ decoder_states)

In [44]:
# 인덱스로부터 단어를 얻는 dictionary 

idx_to_src = dict((i, char) for char, i in src_to_idx.items())  
idx_to_tar = dict((i, char) for char, i in tar_to_idx.items()) 

In [45]:
import numpy as np

def predict_decode(input_seq):
    # 입력으로부터 인코더의 상태얻음
    outputs_input, states_value = encoder_model.predict(input_seq)  
    

    # <SOS>에 해당하는 원-핫 벡터 생성 
    target_seq = np.zeros((1,1,tar_vocab_size))  
    target_seq[0,0,tar_to_idx['\t']] = 1  
    

    stop = False  
    decoded_sentence = ""  

    # stop이 True가 될때까지 루프 반복 = 즉, end될때까지 번역하는 반복문 
    while not stop:
        output_tokens, h, c = decoder_model.predict([target_seq,states_value[0], outputs_input] + states_value)  
        
        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])  
        sampled_char = idx_to_tar[sampled_token_index]  
        
        # 현재 시점의 예측 문자를 예측 문장에 추가 
        decoded_sentence += sampled_char  
        
        # <EOS> 에 도달하거나 최대 길이를 넘으면 중단.
        if sampled_char == '\n' or len(decoded_sentence) > max_tar_len:  
            stop = True  
            
        # 현재 시점의 예측결과를 다음 시점의 입력으로 사용하기 위해 저장 
        target_seq = np.zeros((1,1, tar_vocab_size))  
        target_seq[0,0,sampled_token_index] =1.
        states_value = [h,c]  
    return decoded_sentence  
    

In [46]:
for seq_index in [39098, 29027, 32825, 27062, 39277]: # 입력 문장의 인덱스 무작위 선정 
    input_seq = encoder_input[seq_index : seq_index+1]  
    decoded_sentence = predict_decode(input_seq)  
    
    print('입력:' , lines.src[seq_index])  
    print('정답:' , lines.tar[seq_index][1:len(lines.tar[seq_index])-1])  # '\t'와 '\n' 을 뺴고 출력 
    print('번역:' , decoded_sentence[:len(decoded_sentence)-1], '\n')  # '\n'을 뺴고 출력 

입력: You're a clever kid.
정답: Tu es un enfant intelligent.
번역: Vous êtes un bon garçon. 

입력: I know your father.
정답: Je connais ton père.
번역: Je sais que tu es intelligent. 

입력: You look very pale.
정답: Vous avez l'air très pâle.
번역: Tu as l'air déprimée. 

입력: You never gave up.
정답: Vous n'avez jamais abandonné.
번역: Vous avez été impressionnée. 

입력: Your friend is here.
정답: Votre amie est ici.
번역: Ton amie est deux foulles. 





*   seq2seq와 동일한 문장을 번역
*   seq2seq에 비해 좀 더 나은 성능을 보여줌

